# Loading

In [1]:
import json
import random
import sys
import bm25
import numpy as np
from tqdm import tqdm

In [2]:
datasets = []

for t in ['train', 'val', 'test']:
    with open('../../dataset/English/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
        print(len(pieces))
        datasets.append(pieces)

3900
1294
1289


In [3]:
datasets[-1][-10]

{'content': 'We’ve got close to 7 million Americans who have access to health care for the first time because of Medicaid expansion.',
 'time': '2014-02-20 00:00:00',
 'label': 'fake',
 'source': 'MFC'}

In [4]:
with open('../../dataset/English/articles/articles.json', 'r') as f:
    raw_articles = json.load(f)
len(raw_articles)

8084

In [5]:
random.sample(raw_articles, 1)[0]

{'date': '2018-05-26',
 'url': 'https://www.snopes.com/fact-check/tim-tebow-kneel-anthem/',
 'title': 'Did Tim Tebow Kneel During the National Anthem in Protest of Abortion?',
 'subtitle': 'Tim Tebow and Colin Kaepernick are both famous for kneeling during football games, but in dramatically different contexts.',
 'claim': 'Former NFL quarterback Tim Tebow knelt on the field during the national anthem as a protest against abortion.',
 'content': 'Former NFL quarterback Colin Kaepernick (“former” because he is a free agent currently unsigned by any NFL team) is most famous among football fans — and a good many people who don’t follow football at all — as the one who kicked off a trend among NFL players in 2016 of refusing to stand to attention during the playing of the American national anthem prior to the start of games as a form of protest: Kaepernick decided to either remain seated or kneel during renditions of the Star Spangled Banner in support of Black Lives Matter and to protest 

In [6]:
corpus = [' '.join([a['title'], a['subtitle'], a['claim'], a['content']])
          for a in raw_articles]
len(corpus)

8084

In [7]:
random.sample(corpus, 1)[0]

"Did Mitch McConnell Vow to Block the Appointment of a Russia Investigation Special Prosecutor? Reports grossly misrepresent the Senate Majority Leader's actual statements. Mitch McConnell said he would block the appointment of a special counsel to investigate Russian election interference On 10 May 2017, Addicting Info reported that Republican Senate Majority Leader Mitch McConnell had vowed to\xa0block the appointment of an independent counsel\xa0to investigate Russian efforts to interfere in the United States’ 2016 elections, along with\xa0alleged collusion between the Trump campaign team and Russia: Siding with Trump’s Attorney General Jeff Sessions and Trump’s spokesperson, McConnell told America from the floor of the Senate that he sees no reason to assign an independent special prosecutor or create a bipartisan committee to get to the bottom of Trump’s relationship to Russia. The story was published the day after President Donald Trump fired Federal Bureau of Investigation direc

In [8]:
model = bm25.init_model(corpus)

## Example

In [18]:
p = random.sample(random.sample(datasets, 1)[0], 1)[0]
print(p['content'], p['label'], p['time'])
print()

scores = bm25.get_a_query_scores(p['content'], model)
scores, scores.shape

for i, idx in enumerate(scores.argsort()[::-1][:10]):
    print('='*10, 'Top:',i, ', Score =', scores[idx], '='*10, '\n')
    print(raw_articles[idx]['date'])
    print(corpus[idx])
    print()

Every Republican Senate candidate has now announced whether or not they support the Senate taking up the Supreme Court vacancy…except for Congressman Heck. fake 2016-02-23 00:00:00

========== Top: 0 , Score = 670.6296338984752 ========== 

2004-10-14
American Cancer Society Hoax Will the American Cancer Society donate three cents to cancer research for every e-mail forwarded? For every forwarded copy of an e-mail received, the American Cancer Society will donate three cents to cancer research. It sounds too easy to be true: forward an e-mail, rack up some cancer research money. And it is. Too easy to be true, that is. This “dying child chain letter” hoax now circulating on the net victimizes the American Cancer Society. In the name of a fictitious little girl, people are exhorted to forward the letter on because each forward drops more money into the research coffers. Such an offer is hard to resist because it’s a painless good deed, a way to enjoy a self-congratulatory pat on the bac

## Total

In [19]:
for i, pieces in enumerate(datasets):
    queries = [p['content'] for p in pieces]
    scores_arr = bm25.get_total_scores(queries, corpus, bm25_model=model)
    np.save(
        'data/English/bm25_{}.npy'.format(['train', 'val', 'test'][i]), scores_arr)

  0%|          | 0/1420 [00:00<?, ?it/s]

bm25_scores:  (4256, 8084)


  0%|          | 0/1420 [00:00<?, ?it/s]

bm25_scores:  (1420, 8084)


100%|██████████| 1420/1420 [04:56<00:00,  4.79it/s]

bm25_scores:  (1420, 8084)


# Export Top 10 (only articles published BEFORE the post)

## Loading

In [1]:
import json
import random
import sys
import bm25
import numpy as np
from tqdm import tqdm
import numpy as np
import pickle

In [2]:
datasets = []

for t in ['train', 'val', 'test']:
    with open('../../dataset/English/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
        print(len(pieces))
        datasets.append(pieces)

4256
1420
1420


In [3]:
datasets[-1][-10]

{'content': 'California To Register Illegal Aliens To Vote – Automatically.',
 'time': '2018-01-21 00:00:00',
 'label': 'fake',
 'source': 'MFC'}

In [4]:
with open('../../dataset/Twitter/articles/Snopes_8084.json', 'r') as f:
    raw_articles = json.load(f)
len(raw_articles)

8084

In [5]:
random.sample(raw_articles, 1)[0]

{'date': '2019-03-29',
 'url': 'https://www.snopes.com/fact-check/crystal-methodist-church-sign/',
 'title': 'Is this Crystal Methodist Church Sign in Effing, SC, Real?',
 'subtitle': 'This fictional town certainly seems like an interesting place to live.',
 'claim': 'A photograph shows a church sign outside of Crystal Methodist in Effing, South Carolina, that reads "Adultery is a sin. You can\'t have your Kate and Edith too."',
 'content': 'In March 2019, we started receiving queries about a church sign supposedly found outside of the Crystal Methodist Church in Effing, South Carolina, reading, “Adultery is a sin. You can’t have you Kate and Edith Too”:  This image racked up tens of thousands of shares as it was posted by a variety of Facebook pages, such as “ IntelligentRedneck ” and “ Completely Offensively,” but it originated on the “ Crystal Methodist Church of Effing SC” Facebook page in February 2019. While this image contains a few obvious punny jokes (Crystal Methodist Church 

## Find the nearest article

In [6]:
articles_times = [a['date'] for a in raw_articles]
len(articles_times)

8084

In [7]:
articles_times

['1995-09-25',
 '1995-10-19',
 '1996-02-26',
 '1996-02-27',
 '1996-03-01',
 '1996-07-29',
 '1996-08-02',
 '1996-08-06',
 '1996-08-06',
 '1996-08-11',
 '1996-10-02',
 '1996-12-04',
 '1996-12-13',
 '1996-12-29',
 '1996-12-31',
 '1997-02-09',
 '1997-04-20',
 '1997-04-20',
 '1997-04-24',
 '1997-05-05',
 '1997-06-02',
 '1997-06-03',
 '1997-06-09',
 '1997-07-11',
 '1997-07-12',
 '1997-07-12',
 '1997-07-12',
 '1997-07-26',
 '1997-09-02',
 '1997-10-25',
 '1997-11-02',
 '1997-12-13',
 '1997-12-14',
 '1998-01-01',
 '1998-01-24',
 '1998-02-15',
 '1998-03-01',
 '1998-03-03',
 '1998-03-12',
 '1998-03-29',
 '1998-06-08',
 '1998-06-09',
 '1998-07-01',
 '1998-08-19',
 '1998-08-28',
 '1998-09-01',
 '1998-09-16',
 '1998-09-18',
 '1998-09-28',
 '1998-10-14',
 '1998-10-20',
 '1998-10-28',
 '1998-10-29',
 '1998-11-30',
 '1998-11-30',
 '1998-12-03',
 '1998-12-05',
 '1998-12-05',
 '1998-12-07',
 '1998-12-10',
 '1998-12-30',
 '1998-12-31',
 '1998-12-31',
 '1998-12-31',
 '1998-12-31',
 '1998-12-31',
 '1998-12-

In [9]:
'2004-08-19 16:59:00' < '2005-08-12', '2004-08-19 16:59:00' > '1995-09-25'

(True, True)

In [10]:
def find_the_nearest_article_idx(post_time):
    assert len(articles_times) == len(raw_articles)
    times = articles_times.copy()
    times.append(post_time)
    times.sort()
    return times.index(post_time) - 1

In [16]:
p = random.sample(random.sample(datasets, 1)[0], 1)[0]
print(p['content'], '\n', p['label'], p['time'])
print()

idx = find_the_nearest_article_idx(p['time'])

print(articles_times[idx])
print()
print(articles_times[idx-5:idx+5])

@_celia_marie_ Hey Bernsters, remember when Bernie used this picture in his campaign ads? The ad claimed this was Bernie speaking before a civil rights protest.  He lied.  This is not him.  
 real 2016-03-03 00:00:00

2016-03-03

['2016-03-03', '2016-03-03', '2016-03-03', '2016-03-03', '2016-03-03', '2016-03-03', '2016-03-04', '2016-03-04', '2016-03-04', '2016-03-04']


## Export

### Example

In [17]:
bm25_rank = np.load('./data/English/bm25_train.npy')
bm25_rank.shape

(4256, 8084)

In [18]:
bm25_rank

array([[474.5582239 , 478.1189799 , 473.801308  , ..., 476.66091513,
        455.84552223, 477.47512737],
       [734.29321683, 733.0861948 , 722.22300744, ..., 735.1982627 ,
        710.01138916, 731.75556847],
       [491.92099882, 486.22457925, 480.67503881, ..., 489.57863723,
        465.94916825, 491.06344999],
       ...,
       [420.94519845, 415.27990024, 404.62425888, ..., 421.38729295,
        400.98557363, 419.1878937 ],
       [503.38146788, 503.1384529 , 495.81510119, ..., 500.31376117,
        480.06735291, 499.23994121],
       [430.3317642 , 426.94202539, 425.91989209, ..., 434.47412537,
        411.19915461, 430.74802991]])

In [19]:
ranked_arr = (-bm25_rank).argsort()

In [20]:
ranked_arr

array([[3399, 1210,  734, ..., 2516, 4978, 1033],
       [6074,  425, 6412, ..., 2516, 1033, 4978],
       [6707, 6635, 6303, ..., 1033, 1254, 4978],
       ...,
       [2225, 3577, 3435, ..., 1033, 5260, 4978],
       [1175, 3659, 2799, ..., 2516, 1033, 4978],
       [3610, 5397,  798, ..., 1033, 1254, 4978]])

In [21]:
bm25_rank[0][3399],bm25_rank[0][1210],bm25_rank[0][4978],bm25_rank[0][1033],

(501.686076282779, 500.8389143295417, 431.7864262599631, 416.0683699264424)

In [22]:
datasets[0][0]

{'content': 'On Monday, Paul George was back shooting hoops in the gym. \n\nYesterday, practice became a 1-on-1 game vs. his dog: ',
 'time': '2014-10-22 15:10:15',
 'label': 'real',
 'source': 'twitter'}

In [23]:
raw_articles[3399]

{'date': '2016-09-22',
 'url': 'https://www.snopes.com/fact-check/donald-trump-born-in-the-philippines/',
 'title': 'Was Donald Trump Born in the Philippines?',
 'subtitle': 'A satirical poem by Garrison Keillor mocking conspiracy theories led some readers to believe that Donald Trump was born in the Philippines.',
 'claim': 'Donald Trump was born in the Philippines, and not Queens, New York, as he claims.',
 'content': 'On 21 September 2016, the Washington Post  published a satirical poem by humorist Garrison Keillor that included a line stating Republican presidential nominee Donald Trump was born in the city of Manila in the Phillippines, and not in Queens, New York: Donald Trump was not born in Queens,\nHe was born in the Philippines,\nIn a hotel in downtown Manila.\nWhere his hair turned bright vanilla The poem was not preceded or followed by any disclaimers labeling it as satire, leaving some readers fooled into thinking that Keillor was making a bold declaration about Trump’s no

In [24]:
idx = find_the_nearest_article_idx(datasets[0][0]['time'])
idx

1449

In [25]:
ranked_arr[0]

array([3399, 1210,  734, ..., 2516, 4978, 1033])

In [26]:
valid_idxs = np.where(ranked_arr[0] <= idx, ranked_arr[0], -1)
valid_idxs = [i for i in valid_idxs.tolist() if i != -1]
valid_idxs[:10]

[1210, 734, 506, 1323, 546, 365, 309, 579, 798, 1003]

In [27]:
raw_articles[1210]

{'date': '2012-12-23',
 'url': 'https://www.snopes.com/fact-check/track-meet/',
 'title': 'Why NORAD Tracks Santa',
 'subtitle': "The story of how NORAD came to track Santa's progress every Christmas Eve.",
 'claim': "NORAD annually tracks Santa's progress because a child trying to reach Santa via a newspaper advertisement called the wrong phone number.",
 'content': "Ever since the mid-1950s, generations of children in North America (and, since the advent of the Internet, children from all over the world) have eagerly turned to an annual service provided by the U.S.-Canada North American Aerospace Defense Command ( NORAD ) to help them track the progress of Santa Claus every Christmas Eve as he departs the North Pole and traverses the globe in his reindeer-driven sleigh, delivering presents to good little boys and girls around the world. NORAD’s engaging well over a thousand people to provide a yearly Santa-tracking program seems to many like a rather whimsical venture for a staid def

### Total

In [28]:
for t in ['train', 'val', 'test']:
    with open('../../dataset/English/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
    print('Pieces: ', len(pieces))
    
    bm25_rank = np.load('./data/English/bm25_{}.npy'.format(t))
    print(t, bm25_rank.shape)
    
    print('\nSorting...')
    ranked_arr = (-bm25_rank).argsort()
    print('Done. ranked_arr: {}\n'.format(ranked_arr.shape))
    
    ranked_articles = []
    zero_num = 0
    for i, p in enumerate(tqdm(pieces)):
        idx = find_the_nearest_article_idx(p['time'])
        
        valid_idxs = np.where(ranked_arr[i] <= idx, ranked_arr[i], -1)
        valid_idxs = [aid for aid in valid_idxs.tolist() if aid != -1]
        if len(valid_idxs) == 0:
            zero_num += 1
        
        ranked_articles.append(valid_idxs)
        
    with open('./data/English/ranked_articles_{}.pkl'.format(t), 'wb') as f:
        pickle.dump(ranked_articles, f)
        
    print('Dataset: {}, zero_num = {}\n{}\n'.format(t, zero_num, '='*15))

Pieces:  4256
train (4256, 8084)

Sorting...


  4%|▍         | 173/4256 [00:00<00:02, 1705.02it/s]

Done. ranked_arr: (4256, 8084)



100%|██████████| 4256/4256 [00:02<00:00, 1631.07it/s]


Dataset: train, zero_num = 0

Pieces:  1420
val (1420, 8084)

Sorting...


...

Dataset: val, zero_num = 0

Pieces:  1420
test (1420, 8084)

Sorting...


 13%|█▎        | 178/1420 [00:00<00:00, 1774.38it/s]

Done. ranked_arr: (1420, 8084)



100%|██████████| 1420/1420 [00:00<00:00, 1698.89it/s]


Dataset: test, zero_num = 0



### Top10

In [1]:
import pickle
import random
import numpy as np
from tqdm import tqdm

In [2]:
TOP=10

for t in ['train', 'val', 'test']:
    with open('./data/English/ranked_articles_{}.pkl'.format(t), 'rb') as f:
        ranked_articles = pickle.load(f)
    print('{}: {}'.format(t, len(ranked_articles)))
    
    # <Padding> value = -1
    top_arr = np.zeros((len(ranked_articles), TOP)) - 1
    for i, ranks in enumerate(tqdm(ranked_articles)):
        sz = min(TOP, len(ranks))
        top_arr[i, :sz] = ranks[:sz]
        
    file = './data/English/Top{}_articles_{}.npy'.format(TOP, t)
    np.save(file, top_arr)

100%|██████████| 4256/4256 [00:00<00:00, 361209.18it/s]

train: 4256



100%|██████████| 1420/1420 [00:00<00:00, 351754.76it/s]

val: 1420
test: 1420


In [3]:
top_arr.shape

(1420, 10)

In [4]:
top_arr

array([[ 357., 1044.,  927., ..., 1003., 1002.,  603.],
       [ 706., 1806., 2548., ..., 3629., 1179., 1894.],
       [ 603.,  969., 2206., ...,  991.,  798., 2377.],
       ...,
       [2410., 5322., 3628., ..., 1287., 1841., 2746.],
       [ 192., 1044.,  766., ..., 1563.,  560.,  506.],
       [ 969.,  733.,  991., ...,  793.,  657.,  121.]])

# Check

In [1]:
import json
import random
import sys
import bm25
import numpy as np
from tqdm import tqdm

In [2]:
datasets = []
top_articles = []

for t in ['train', 'val', 'test']:
    with open('../../dataset/English/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
    print(len(pieces))
    
    arr = np.load('./data/English/Top10_articles_{}.npy'.format(t))
    arr = np.array(arr, dtype=int)
    print(arr.shape)
    
    datasets.append(pieces)
    top_articles.append(arr)

3900
(4256, 10)
1294
(1420, 10)
1289
(1420, 10)


In [3]:
with open('../../dataset/English/articles/articles.json', 'r') as f:
    raw_articles = json.load(f)
len(raw_articles)

8084

In [4]:
t = random.randint(0,2)
pieces = datasets[t]
arr = top_articles[t]

i = random.randint(0, len(pieces)-1)
p = pieces[i]
rank = arr[i]

print(p['content'], p['label'], p['time'])
print()

for i, idx in enumerate(rank):
    print('='*20, 'Top:',i, '='*20, '\n')
    print(raw_articles[idx]['date'])
    print(raw_articles[idx]['content'])
    print()

Says Georgia’s 5th congressional district, represented by John Lewis, is "in horrible shape and falling apart (not to mention crime infested)." fake 2017-01-14 00:00:00

==================== Top: 0 ==================== 

2006-01-01
Valentin Mikhaylin, a young man who lives in Kaluga, Russia, and who represents himself as a student starving and on the verge of freezing to death, has been spamhandling on the Internet since 1998. His e-mailed appeals are generally loosed upon the unsuspecting in November of each year, perhaps because the final month before Christmas tends to spark the charitable impulse in the  Please excuse us for any inconveniences we could cause you by our message. As you know, now Russia has a deep financial and economic crisis. We already for some months do not receive our salary. Our parents and invalids also do not receive pensions and forced to hand over blood to buy necessary for live foodstuffs. Because of all it, our children are constantly hungry and ill from 